# Extract SourcesTables Results in  OGA

 work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/01/19
- update : 2023/01/19


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

## Configuration

In [8]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [9]:
file_out="sourceTable_202301-17-18.csv"

## Give the collection

In [10]:
#my_collection = "u/dagoret/BPS_test"
my_collection = "u/dagoret/BPS_photimaging_202301"

In [11]:
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Config

In [12]:
# path index for each month
#DATE="20211103"
#filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

# Sources

In [13]:
datasetRefs = butler.registry.queryDatasets(datasetType='sourceTable', collections=my_collection, where= "instrument='LATISS'")

In [26]:
all_ref_for_src = []
all_dataId = []
all_src = []
all_exposures = []
all_sourcesTables = []

for i, ref in enumerate(datasetRefs):
    all_ref_for_src.append(ref.dataId)
    print("========================datasetType = sourceTable ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_visit = ref.dataId["visit"]
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    print(the_dataId)
    df_tab       =  butler.get(ref,collections=my_collection)
    N=len(df_tab)
    df_tab['visid_idx']= (i*np.ones(N)).astype(int)
    all_sourcesTables.append(df_tab) 
    all_dataId.append(the_dataId) 
    
    #if i > 5:
    #    break

========================datasetType = sourceTable ============================================
fullId..................: {band: 'r', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSr_65mm~empty', visit: 2023011700190}
visit...................: 2023011700190
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: u/dagoret/BPS_photimaging_202301/20230118T172011Z
{'day_obs': 20230117, 'seq_num': 190, 'detector': 0}
========================datasetType = sourceTable ============================================
fullId..................: {band: 'r', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSr_65mm~empty', visit: 2023011700204}
visit...................: 2023011700204
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: u/dagoret/BPS_photimaging_202301/20230118T172011Z
{'day_obs': 20230117, 'seq_num': 204, 'detector': 0}
========================datasetType = sourceTable ================

In [24]:
len(all_sourcesTables)

86

In [25]:
all_sourcesTables[0]

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx
sourceId,,,,,,,,,,,,,,,,,,,,,
2121281516538429441,89.420770,-52.691373,2023011700190,0,1132.000000,29.000000,NaN,NaN,89.420770,-52.691373,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429442,89.442960,-52.691314,2023011700190,0,1638.000000,25.000000,NaN,NaN,89.442960,-52.691314,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429443,89.532580,-52.690340,2023011700190,0,3682.000000,35.000000,NaN,NaN,89.532580,-52.690340,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429444,89.418212,-52.688397,2023011700190,0,1075.009622,141.635676,0.012064,0.012581,89.418212,-52.688397,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429445,89.405873,-52.686825,2023011700190,0,794.351054,204.157401,0.031180,0.030598,89.405873,-52.686825,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121281516538429668,89.377677,-52.593478,2023011700190,2121281516538429527,192.000000,3723.000000,NaN,NaN,89.377677,-52.593478,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429669,89.377401,-52.587965,2023011700190,2121281516538429535,188.074598,3930.446161,1.507361,0.697237,89.377401,-52.587965,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0
2121281516538429670,89.377508,-52.589619,2023011700190,2121281516538429535,189.801940,3868.172822,0.407219,1.041372,89.377508,-52.589619,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0.0


In [28]:
df=pd.concat(all_sourcesTables)

In [29]:
df

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx
sourceId,,,,,,,,,,,,,,,,,,,,,
2121281516538429441,89.420770,-52.691373,2023011700190,0,1132.000000,29.000000,NaN,NaN,89.420770,-52.691373,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0
2121281516538429442,89.442960,-52.691314,2023011700190,0,1638.000000,25.000000,NaN,NaN,89.442960,-52.691314,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0
2121281516538429443,89.532580,-52.690340,2023011700190,0,3682.000000,35.000000,NaN,NaN,89.532580,-52.690340,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0
2121281516538429444,89.418212,-52.688397,2023011700190,0,1075.009622,141.635676,0.012064,0.012581,89.418212,-52.688397,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0
2121281516538429445,89.405873,-52.686825,2023011700190,0,794.351054,204.157401,0.031180,0.030598,89.405873,-52.686825,...,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121281516509072015,13.373763,-72.935386,2023011700162,2121281516509070608,2621.067346,3932.738382,8.041463,3.297168,13.373763,-72.935386,...,False,False,False,False,True,2023011700162,0,r,SDSSr_65mm~empty,85
2121281516509072016,13.372498,-72.934339,2023011700162,2121281516509070608,2607.622876,3972.308543,8.578441,1.188897,13.372498,-72.934339,...,False,False,False,False,True,2023011700162,0,r,SDSSr_65mm~empty,85
2121281516509072017,13.471415,-72.934788,2023011700162,2121281516509070614,3699.000000,3940.000000,NaN,NaN,13.471415,-72.934788,...,False,False,False,False,True,2023011700162,0,r,SDSSr_65mm~empty,85


In [30]:
df.to_csv(file_out)